# Error Generation and Model fitting

This code will add errors to the output of Data Generation.ipynb and iteratively create scenarios, perform model fitting and performance evaluation.

In [ ]:
crowd_data = pd.read_excel(r'crowd/24_person_grid.xlsx')
crowd_data = crowd_data.iloc[:,1:]
df = crowd_data.iloc[:,[1,2,5,6]]
df_cols = list(df.columns)
df_l =[]     #### global library
scenario = {'Intensity':[0] , 'Duration':[1], 'Inten_Dur':[0,1]} ### Scenarios for misclassification
dlist = [1,2,3]
per_error = [0,10,20,30]
classifier = ['RandomForestClassifier','LogisticRegression']
# classifier = ['RandomForestClassifier']
for err in per_error:
    index = np.random.randint(0 , df.shape[0] , size = round(err*df.shape[0]/100))
    for scene in scenario.keys():
        df_err = df.values
        for j in range (len(scenario[scene])):
            y = scenario[scene]
            for i in index:
                if df_err[i,y[j]] == 1:
                    dlist.remove(1)
                    df_err[i,y[j]] = random.choice(dlist)
                elif df_err[i,y[j]] == 2:
                    dlist.remove(2)
                    df_err[i,y[j]] = random.choice(dlist)
                else:
                    dlist.remove(3)
                    df_err[i,y[j]] = random.choice(dlist)
                dlist = [1,2,3]
        features = df_err[:,0:3]
        target   = df_err[:,3]
        X_train,X_test,Y_train,Y_test = train_test_split(features, target, test_size = 0.3,random_state =42)
        for clf in classifier:
            sub_list = []
            if clf =='RandomForestClassifier':
                clf_p = eval(clf)(criterion='gini',random_state = 0)
            elif clf =='LogisticRegression':
                clf_p = eval(clf)(multi_class = 'multinomial',random_state = 0,max_iter=1000)
            else:
                clf_p = eval(clf)(random_state=42)
            clf_p.fit(X_train,Y_train)
            Y_train_predict,Y_test_predict = clf_p.predict(X_train),clf_p.predict(X_test)
            conf_matrix = confusion_matrix(Y_test_predict, Y_test)
            scores = cross_val_score(clf_p, X_train, Y_train, cv = 3, scoring='accuracy')
            print(f'Cross val accuracy for {clf} = ', scores.mean()*100,'%')
#             sns.heatmap(conf_matrix)
            #print(conf_matrix)
            conf_mat1,conf_mat2,conf_mat3   = np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
            conf_mat1[0,0],conf_mat1[0,1],conf_mat1[1,0],conf_mat1[1,1] = conf_matrix[0,0],conf_matrix[0,1]+conf_matrix[0,2],\
                                                           conf_matrix[1,0]+conf_matrix[2,0],conf_matrix[1,1]+\
                                                           conf_matrix[1,2]+conf_matrix[2,1]+conf_matrix[2,2]
            conf_mat2[0,0],conf_mat2[0,1],conf_mat2[1,0],conf_mat2[1,1] = conf_matrix[1,1],conf_matrix[1,0]+conf_matrix[1,2],\
                                                                         conf_matrix[0,1]+conf_matrix[2,1],conf_matrix[0,0]+\
                                                                         conf_matrix[0,2]+conf_matrix[2,0]+conf_matrix[2,2]
            conf_mat3[0,0],conf_mat3[0,1],conf_mat3[1,0],conf_mat3[1,1] = conf_matrix[2,2],conf_matrix[2,0]+conf_matrix[2,1],\
                                                                          conf_matrix[0,2]+conf_matrix[1,2],conf_matrix[0,0]+\
                                                                          conf_matrix[1,1]+conf_matrix[0,1]+conf_matrix[1,0]
#             print(conf_mat3)
            TS = (conf_mat3[0,0])/(conf_mat3[0,0]+conf_mat3[0,1]+conf_mat3[1,0])
            FAR = (conf_matrix[0,1])/(conf_matrix[0,1]+conf_matrix[1,1])
            accuracy = accuracy_score(Y_test, Y_test_predict)
            sigmysigmo =(np.sum(conf_matrix,axis=1)[0]*np.sum(conf_matrix,axis=0)[0]+np.sum(conf_matrix,axis=1)[1]*np.sum(conf_matrix,axis=0)[1]\
                         +np.sum(conf_matrix,axis=1)[2]*np.sum(conf_matrix,axis=0)[2])\
                        /(len(Y_test)**2)
            HSS = (accuracy-sigmysigmo)/(1-sigmysigmo)
            sub_list =[err,scene,clf,scores.mean()*100,HSS,TS,FAR]
            df_l.append(sub_list)
            
            
            
                

## Performance Evaluation with spatial information

In [ ]:
#### Importing Libraries
import numpy as np
import pandas as pd
#### Calling Data
crowd_data = pd.read_csv(r'/Volumes/One_Touch/Project/crowd_sourcing/crowd/5_person_grid(1).csv')
crowd_data = crowd_data.iloc[:,1:]
df = crowd_data.iloc[:,[0,1,2,3,6,7]]
df_cols = list(df.columns)
df.insert(loc = 1 ,column = 'Person_id', value = crowd_data['Person_id'].values)
df_cols = list(df.columns)
#### Adding errors or scenario generation
import random
scene = {'Intensity':[3] , 'Duration':[4], 'Inten_Dur':[3,4]}
dlist = [1.0,2.0,3.0]
perc_error = 30
index = np.random.randint(0 , df.shape[0] , size = round(perc_error*df.shape[0]/100))
df_err = df.values
x = 'Inten_Dur'
for j in range (len(scene[x])):
    y = scene[x]
    for i in index:
        if df_err[i,y[j]] == 1:
            dlist.remove(1)
            df_err[i,y[j]] = random.choice(dlist)
        elif df_err[i,y[j]] == 2:
            dlist.remove(2)
            df_err[i,y[j]] = random.choice(dlist)
        else:
            dlist.remove(3)
            df_err[i,y[j]] = random.choice(dlist)
        dlist = [1,2,3]
err_df = pd.DataFrame(df_err,columns = df_cols)

err_df['Target'] = err_df['Target'].astype('int')
input_feats = ['Gauge_Id','Intensity_class','Duration_class','Storm Center']  #### Using spatial information as feature
features = err_df[input_feats]
target = 'Target'
target_feats = err_df[target]
##### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(features, target_feats, test_size = 0.3,random_state =42)
#Y_train = Y_train.astype('int32')

#### Importing Libraries for Accuracy check
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix


#### Importing Libraries For Model Fitting

from sklearn.ensemble import RandomForestClassifier      #### Random Forest
from sklearn.linear_model import LogisticRegression      #### Logistic Regression

#### Random Forest Ensemble Model

print("Random Forest Results")
rnd_model = RandomForestClassifier(criterion='gini',random_state = 0)
rnd_model.fit(X_train,Y_train)
Y_train_predict = rnd_model.predict(X_train)
Y_test_predict  = rnd_model.predict(X_test)
print('Training accuracy got RF = ',accuracy_score(Y_train, Y_train_predict)*100,'%')
print('Test accuracy for RF = ',accuracy_score(Y_test, Y_test_predict)*100,'%')
scores = cross_val_score(rnd_model, X_train, Y_train, cv = 3, scoring='accuracy')
print('Cross val accuracy for RF = ', scores.mean()*100,'%')
print('Confusion Matrix for Random Forest: \n\n', confusion_matrix(Y_train, Y_train_predict))
print('\n############################################')


#### Logistic Regression Classifier

print("Logistic Regression Results")
model = LogisticRegression(multi_class = 'multinomial',random_state = 0,max_iter=1000)
model.fit(X_train,Y_train)
Y_train_predict = model.predict(X_train)
Y_test_predict  = model.predict(X_test)
print('Training accuracy for LG = ',accuracy_score(Y_train, Y_train_predict)*100,'%')
print('Test accuracy for LG = ',accuracy_score(Y_test, Y_test_predict)*100,'%')
scores = cross_val_score(model, X_train, Y_train, cv = 3, scoring='accuracy')
print('Cross val accuracy for LG = ', scores.mean()*100,'%')
print('Confusion Matrix for Logistic Regression: \n\n', confusion_matrix(Y_train, Y_train_predict))

## Feature Importance For Random Forest Model

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
importances = rnd_model.feature_importances_
feature_names = features.columns
feature_importance_df = pd.DataFrame({
    'feature': ['Duration','Intensity'],
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Display feature importances
print("Feature Importances:\n", feature_importance_df)

# Plot feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance_df)
plt.title('Feature Importances')
plt.show()